In [14]:
# 1. CARREGAMENTO DOS MODELOS TREINADOS
from utils import *
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os
import pickle
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações
config = carregar_configuracoes()
print("=== CARREGANDO MODELOS TREINADOS ===")

# Carregar modelo de classificação de espécies
print("📂 Carregando modelo de espécies...")
modelo_especies = load_model('modelos_salvos/melhor_modelo_especies_final_otimizado.h5')
with open('datasets_processados/label_encoder_especies_modelo.pkl', 'rb') as f:
    encoder_especies = pickle.load(f)

print(f"   ✅ Modelo de espécies: {encoder_especies.classes_}")

# Carregar modelos de classificação de saúde
print("📂 Carregando modelos especialistas...")
modelos_especialistas = {}
especies_disponiveis = []

for especie in ['tomato', 'potato', 'pepper']:
    modelo_path = f'modelos_salvos/especialistas/especialista_{especie}_binario_final.h5'
    if os.path.exists(modelo_path):
        modelos_especialistas[especie] = load_model(modelo_path)
        especies_disponiveis.append(especie)
        print(f"   ✅ Modelo {especie}: carregado")
    else:
        print(f"   ⚠️ Modelo {especie}: não encontrado")


=== CARREGANDO MODELOS TREINADOS ===
📂 Carregando modelo de espécies...
   ✅ Modelo de espécies: ['Pepper_bell' 'Potato' 'Tomato']
📂 Carregando modelos especialistas...
   ✅ Modelo tomato: carregado
   ✅ Modelo potato: carregado
   ✅ Modelo pepper: carregado


In [19]:
# 2. IMPLEMENTAÇÃO DO PIPELINE HIERÁRQUICO
def preprocessar_imagem(caminho_imagem, target_size=(224, 224)):
    """Preprocessa imagem para os modelos"""
    img = image.load_img(caminho_imagem, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def pipeline_hierarquico(caminho_imagem, modelo_especies, encoder_especies, modelos_especialistas, threshold_confianca=0.5):
    """
    Pipeline completo: Passo 3 (espécie) → Passo 4 (saúde)
    
    Returns:
        dict: {
            'especie_predita': str,
            'confianca_especie': float,
            'saude_predita': str,
            'confianca_saude': float,
            'resultado_final': str,
            'confianca_final': float,
            'etapas': dict
        }
    """
    # Preprocessar imagem
    img_preprocessada = preprocessar_imagem(caminho_imagem)
    
    #Classificar espécie
    pred_especies = modelo_especies.predict(img_preprocessada, verbose=0)
    indice_especie = np.argmax(pred_especies)
    especie_predita = encoder_especies.inverse_transform([indice_especie])[0]
    confianca_especie = np.max(pred_especies)
    
    # Mapear nome da espécie para o modelo especialista
    mapeamento_especies = {
        'Tomato': 'tomato',
        'Potato': 'potato', 
        'Pepper_bell': 'pepper'
    }
    
    especie_modelo = mapeamento_especies.get(especie_predita)
    
    #Classificar saúde
    if especie_modelo and especie_modelo in modelos_especialistas:
        modelo_especialista = modelos_especialistas[especie_modelo]
        pred_saude = modelo_especialista.predict(img_preprocessada, verbose=0)[0][0]
        
        # Conversão binária: >0.5 = unhealthy, <=0.5 = healthy
        if pred_saude > 0.5:
            saude_predita = 'unhealthy'
            confianca_saude = pred_saude
        else:
            saude_predita = 'healthy'
            confianca_saude = 1 - pred_saude
            
        # Resultado final combinado
        resultado_final = f"{especie_predita}_{saude_predita}"
        confianca_final = confianca_especie * confianca_saude  # Propagação de incerteza
        
        sucesso_pipeline = True
        
    else:
        # Modelo especialista não disponível
        saude_predita = 'unknown'
        confianca_saude = 0.0
        resultado_final = f"{especie_predita}_unknown"
        confianca_final = confianca_especie
        sucesso_pipeline = False
    
    return {
        'especie_predita': especie_predita,
        'confianca_especie': float(confianca_especie),
        'saude_predita': saude_predita,
        'confianca_saude': float(confianca_saude),
        'resultado_final': resultado_final,
        'confianca_final': float(confianca_final),
        'sucesso_pipeline': sucesso_pipeline,
        'etapas': {
            'passo3_especies': {'pred': especie_predita, 'conf': float(confianca_especie)},
            'passo4_saude': {'pred': saude_predita, 'conf': float(confianca_saude)}
        }
    }

print("✅ Pipeline hierárquico implementado!")


# Encontrar uma imagem de teste
test_image = None
for especie in ['Tomato', 'Potato', 'Pepper_bell']:
    for classe in config['especialistas'][especie]['classes']:
        pasta_classe = os.path.join(config['base_path'], classe)
        if os.path.exists(pasta_classe):
            images = [f for f in os.listdir(pasta_classe) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if images:
                test_image = os.path.join(pasta_classe, images[0])
                test_class = classe
                break
    if test_image:
        break

if test_image:
    print(f"\n🧪 Testando com: {test_class}")
    resultado = pipeline_hierarquico(test_image, modelo_especies, encoder_especies, modelos_especialistas)
    print(f"   Espécie: {resultado['especie_predita']} (conf: {resultado['confianca_especie']:.3f})")
    print(f"   Saúde: {resultado['saude_predita']} (conf: {resultado['confianca_saude']:.3f})")
    print(f"   Final: {resultado['resultado_final']} (conf: {resultado['confianca_final']:.3f})")
else:
    print("⚠️ Nenhuma imagem de teste encontrada")


✅ Pipeline hierárquico implementado!

🧪 Testando com: Tomato_Bacterial_spot
   Espécie: Tomato (conf: 0.968)
   Saúde: unhealthy (conf: 1.000)
   Final: Tomato_unhealthy (conf: 0.968)


In [ ]:
# 3. CRIAÇÃO DO CONJUNTO DE TESTE HIERÁRQUICO
def criar_conjunto_teste_hierarquico(config, n_samples_per_class=50):
    """
    Cria conjunto de teste balanceado para avaliação hierárquica
    """
    
    conjunto_teste = []
    
    for especie, info in config['especialistas'].items():
        print(f"📂 Processando {especie}...")
        
        for classe in info['classes']:
            pasta_classe = os.path.join(config['base_path'], classe)
            
            if not os.path.exists(pasta_classe):
                print(f"   ⚠️ Pasta não encontrada: {classe}")
                continue
                
            # Obter imagens da classe
            images = [f for f in os.listdir(pasta_classe) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            if len(images) == 0:
                print(f"   ⚠️ Nenhuma imagem em: {classe}")
                continue
                
            # Selecionar amostra aleatória
            np.random.seed(42)
            n_select = min(n_samples_per_class, len(images))
            selected_images = np.random.choice(images, n_select, replace=False)
            
            # Determinar rótulos
            especie_label = especie
            saude_label = 'healthy' if 'healthy' in classe.lower() else 'unhealthy'
            combined_label = f"{especie}_{saude_label}"
            
            # Adicionar ao conjunto
            for img_name in selected_images:
                caminho_completo = os.path.join(pasta_classe, img_name)
                conjunto_teste.append({
                    'caminho': caminho_completo,
                    'classe_original': classe,
                    'especie_real': especie_label,
                    'saude_real': saude_label,
                    'combined_real': combined_label
                })
            
            print(f"   ✅ {classe}: {n_select} imagens selecionadas")
    
    # Converter para DataFrame
    df_teste = pd.DataFrame(conjunto_teste)
    
    print(f"\n📊 CONJUNTO DE TESTE CRIADO:")
    print(f"   Total de imagens: {len(df_teste)}")
    print(f"   Espécies: {df_teste['especie_real'].value_counts().to_dict()}")
    print(f"   Saúde: {df_teste['saude_real'].value_counts().to_dict()}")
    
    return df_teste

# Criar conjunto de teste
df_teste_hierarquico = criar_conjunto_teste_hierarquico(config, n_samples_per_class=30)

# Salvar para reutilização
df_teste_hierarquico.to_csv('datasets_processados/conjunto_teste_hierarquico.csv', index=False)
print("\n💾 Conjunto de teste salvo em: datasets_processados/conjunto_teste_hierarquico.csv")


📂 Processando Tomato...
   ✅ Tomato_Bacterial_spot: 30 imagens selecionadas
   ✅ Tomato_Early_blight: 30 imagens selecionadas
   ✅ Tomato_Late_blight: 30 imagens selecionadas
   ✅ Tomato_Leaf_Mold: 30 imagens selecionadas
   ✅ Tomato_Septoria_leaf_spot: 30 imagens selecionadas
   ✅ Tomato_Spider_mites_Two_spotted_spider_mite: 30 imagens selecionadas
   ✅ Tomato__Target_Spot: 30 imagens selecionadas
   ✅ Tomato__Tomato_YellowLeaf__Curl_Virus: 30 imagens selecionadas
   ✅ Tomato__Tomato_mosaic_virus: 30 imagens selecionadas
   ✅ Tomato_healthy: 30 imagens selecionadas
📂 Processando Potato...
   ✅ Potato___Early_blight: 30 imagens selecionadas
   ✅ Potato___Late_blight: 30 imagens selecionadas
   ✅ Potato___healthy: 30 imagens selecionadas
📂 Processando Pepper_bell...
   ✅ Pepper__bell___Bacterial_spot: 30 imagens selecionadas
   ✅ Pepper__bell___healthy: 30 imagens selecionadas

📊 CONJUNTO DE TESTE CRIADO:
   Total de imagens: 450
   Espécies: {'Tomato': 300, 'Potato': 90, 'Pepper_bell':

In [ ]:
# 4. AVALIAÇÃO COMPLETA DO PIPELINE HIERÁRQUICO
def avaliar_pipeline_hierarquico(df_teste, modelo_especies, encoder_especies, modelos_especialistas):
    """
    Avalia o pipeline hierárquico completo
    """
    
    resultados = []
    
    # Processar cada imagem
    for idx, row in df_teste.iterrows():
        if idx % 50 == 0:
            print(f"   Processando {idx+1}/{len(df_teste)}...")
            
        # Aplicar pipeline hierárquico
        resultado = pipeline_hierarquico(
            row['caminho'], 
            modelo_especies, 
            encoder_especies, 
            modelos_especialistas
        )
        
        # Adicionar dados reais
        resultado['especie_real'] = row['especie_real']
        resultado['saude_real'] = row['saude_real']
        resultado['combined_real'] = row['combined_real']
        resultado['classe_original'] = row['classe_original']
        
        # Calcular acertos
        resultado['acerto_especie'] = resultado['especie_predita'] == resultado['especie_real']
        resultado['acerto_saude'] = resultado['saude_predita'] == resultado['saude_real']
        resultado['acerto_combined'] = resultado['resultado_final'] == resultado['combined_real']
        
        resultados.append(resultado)
    
    # Converter para DataFrame
    df_resultados = pd.DataFrame(resultados)
    
    return df_resultados

# Executar avaliação
print("🚀 Iniciando avaliação completa...")
df_resultados = avaliar_pipeline_hierarquico(
    df_teste_hierarquico, 
    modelo_especies, 
    encoder_especies, 
    modelos_especialistas
)

# Salvar resultados
df_resultados.to_csv('datasets_processados/resultados_pipeline_hierarquico.csv', index=False)
print("\n💾 Resultados salvos em: datasets_processados/resultados_pipeline_hierarquico.csv")

print(f"\n✅ Avaliação concluída: {len(df_resultados)} predições realizadas")


🚀 Iniciando avaliação completa...
=== AVALIAÇÃO DO PIPELINE HIERÁRQUICO ===
   Processando 1/450...
   Processando 51/450...
   Processando 101/450...
   Processando 151/450...
   Processando 201/450...
   Processando 251/450...
   Processando 301/450...
   Processando 351/450...
   Processando 401/450...

💾 Resultados salvos em: datasets_processados/resultados_pipeline_hierarquico.csv

✅ Avaliação concluída: 450 predições realizadas


In [24]:
# 5. ANÁLISE DE PROPAGAÇÃO DE ERROS E MÉTRICAS
def analisar_propagacao_erros(df_resultados):
    """
    Analisa como erros se propagam no pipeline hierárquico
    """
    
    # Métricas por etapa
    acc_especie = df_resultados['acerto_especie'].mean()
    acc_saude = df_resultados['acerto_saude'].mean()
    acc_combined = df_resultados['acerto_combined'].mean()
    
    print("📊 ACURÁCIAS POR ETAPA:")
    print(f"Classificação de espécie: {acc_especie:.4f} ({acc_especie*100:.1f}%)")
    print(f"Classificação de saúde: {acc_saude:.4f} ({acc_saude*100:.1f}%)")
    print(f"Pipeline completo: {acc_combined:.4f} ({acc_combined*100:.1f}%)")
    
    # Análise de propagação de erros
    print(f"\n🔍 ANÁLISE DE PROPAGAÇÃO:")
    
    # Casos onde espécie está correta
    correto_especie = df_resultados[df_resultados['acerto_especie'] == True]
    if len(correto_especie) > 0:
        acc_saude_dado_especie_correta = correto_especie['acerto_saude'].mean()
        print(f"   Acurácia de saúde QUANDO espécie está correta: {acc_saude_dado_especie_correta:.4f} ({acc_saude_dado_especie_correta*100:.1f}%)")
    
    # Casos onde espécie está incorreta
    incorreto_especie = df_resultados[df_resultados['acerto_especie'] == False]
    if len(incorreto_especie) > 0:
        acc_saude_dado_especie_incorreta = incorreto_especie['acerto_saude'].mean()
        print(f"   Acurácia de saúde QUANDO espécie está incorreta: {acc_saude_dado_especie_incorreta:.4f} ({acc_saude_dado_especie_incorreta*100:.1f}%)")
    
    # Impacto da propagação
    acc_teorica = acc_especie * acc_saude_dado_especie_correta if len(correto_especie) > 0 else 0
    impacto_propagacao = acc_combined - acc_teorica
    
    print(f"\n⚖️ IMPACTO DA PROPAGAÇÃO:")
    print(f"   Acurácia teórica (independente): {acc_teorica:.4f}")
    print(f"   Acurácia real (hierárquica): {acc_combined:.4f}")
    print(f"   Impacto da propagação: {impacto_propagacao:+.4f}")
    
    # Matriz de confusão por etapa
    print(f"\n📈 MATRIZES DE CONFUSÃO:")
    
    # Espécie
    print(f"\n🌱 CLASSIFICAÇÃO DE ESPÉCIE:")
    especies_reais = df_resultados['especie_real'].values
    especies_pred = df_resultados['especie_predita'].values
    print(classification_report(especies_reais, especies_pred, zero_division=0))
    
    # Saúde
    print(f"\n🏥 CLASSIFICAÇÃO DE SAÚDE:")
    saude_reais = df_resultados['saude_real'].values
    saude_pred = df_resultados['saude_predita'].values
    print(classification_report(saude_reais, saude_pred, zero_division=0))
    
    # Análise por espécie
    print(f"\n🔬 ANÁLISE POR ESPÉCIE:")
    for especie in df_resultados['especie_real'].unique():
        subset = df_resultados[df_resultados['especie_real'] == especie]
        acc_esp = subset['acerto_especie'].mean()
        acc_sau = subset['acerto_saude'].mean()
        acc_comb = subset['acerto_combined'].mean()
        
        print(f"   {especie}:")
        print(f"     Espécie: {acc_esp:.3f} | Saúde: {acc_sau:.3f} | Completo: {acc_comb:.3f}")
    
    # Confiança média
    print(f"\n🎯 ANÁLISE DE CONFIANÇA:")
    conf_especie_media = df_resultados['confianca_especie'].mean()
    conf_saude_media = df_resultados['confianca_saude'].mean()
    conf_final_media = df_resultados['confianca_final'].mean()
    
    print(f"   Confiança média - Espécie: {conf_especie_media:.3f}")
    print(f"   Confiança média - Saúde: {conf_saude_media:.3f}")
    print(f"   Confiança média - Final: {conf_final_media:.3f}")
    
    return {
        'acc_especie': acc_especie,
        'acc_saude': acc_saude,
        'acc_combined': acc_combined,
        'acc_teorica': acc_teorica,
        'impacto_propagacao': impacto_propagacao,
        'conf_especie_media': conf_especie_media,
        'conf_saude_media': conf_saude_media,
        'conf_final_media': conf_final_media
    }

# Executar análise
metricas = analisar_propagacao_erros(df_resultados)


📊 ACURÁCIAS POR ETAPA:
Classificação de espécie: 0.9178 (91.8%)
Classificação de saúde: 0.8578 (85.8%)
Pipeline completo: 0.8000 (80.0%)

🔍 ANÁLISE DE PROPAGAÇÃO:
   Acurácia de saúde QUANDO espécie está correta: 0.8717 (87.2%)
   Acurácia de saúde QUANDO espécie está incorreta: 0.7027 (70.3%)

⚖️ IMPACTO DA PROPAGAÇÃO:
   Acurácia teórica (independente): 0.8000
   Acurácia real (hierárquica): 0.8000
   Impacto da propagação: +0.0000

📈 MATRIZES DE CONFUSÃO:

🌱 CLASSIFICAÇÃO DE ESPÉCIE:
              precision    recall  f1-score   support

 Pepper_bell       0.88      0.95      0.91        60
      Potato       0.90      0.78      0.83        90
      Tomato       0.93      0.95      0.94       300

    accuracy                           0.92       450
   macro avg       0.90      0.89      0.90       450
weighted avg       0.92      0.92      0.92       450


🏥 CLASSIFICAÇÃO DE SAÚDE:
              precision    recall  f1-score   support

     healthy       0.68      0.54      0.60  

In [34]:
# 7. COMPARAÇÃO FINAL E CONCLUSÕES
def comparacao_final(metricas, acc_baseline):
    """
    Compara pipeline hierárquico com baseline direto
    """
    
    acc_hierarquico = metricas['acc_combined']
    diferenca = acc_hierarquico - acc_baseline
    
    print("📊 RESULTADOS FINAIS:")
    print(f"   🔗 Pipeline Hierárquico: {acc_hierarquico:.4f} ({acc_hierarquico*100:.1f}%)")
    print(f"   📍 Abordagem Direta: {acc_baseline:.4f} ({acc_baseline*100:.1f}%)")
    print(f"   📈 Diferença: {diferenca:+.4f} ({diferenca*100:+.1f}%)")
    
    return {
        'acc_hierarquico': acc_hierarquico,
        'acc_baseline': acc_baseline,
        'diferenca': diferenca,
    }

# Executar comparação final
comparacao = comparacao_final(metricas, acc_baseline)



📊 RESULTADOS FINAIS:
   🔗 Pipeline Hierárquico: 0.8000 (80.0%)
   📍 Abordagem Direta: 0.8000 (80.0%)
   📈 Diferença: +0.0000 (+0.0%)
